In [1]:
import os
import sys
project_root_dir = os.path.relpath(os.path.join('..', '..'), os.curdir)
if project_root_dir not in sys.path:
    sys.path += [project_root_dir]
from src.data.dataload import *
from src.models.bcn_model import *
from pprint import pprint

ignoring warnings from spacy


# Data<br>
Loading data

In [2]:
data = load_sst()
print(f'loading data {data.NAME} (sentence column: {data.SENTENCE}, target column: {data.TARGET})')
train, val, test = data.train_val_test
train

loading data sst (sentence column: sentence, target column: label)


,sentence,label
0,The Rock is destined to be the 21st Century 's...,3
1,The gorgeously elaborate continuation of `` Th...,4
2,Singer/composer Bryan Adams contributes a slew...,3
3,You 'd think by now America would have had eno...,2
4,Yet the act is still charming here .,3
...,...,...
8539,A real snooze .,0
8540,No surprises .,1
8541,We 've seen the hippie-turned-yuppie plot befo...,3
8542,Her fans walked out muttering words like `` ho...,0


# Model<br>
Fine-tuning and loading BCN

In [3]:
bcn = BCNModel()
bcn.load_model(data)
bcn.model

BiattentiveClassificationNetwork(
  (_text_field_embedder): BasicTextFieldEmbedder(
    (token_embedder_tokens): Embedding()
  )
  (_embedding_dropout): Dropout(p=0.25, inplace=False)
  (_pre_encode_feedforward): FeedForward(
    (_activations): ModuleList(
      (0): ReLU()
    )
    (_linear_layers): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
    )
    (_dropout): ModuleList(
      (0): Dropout(p=0.25, inplace=False)
    )
  )
  (_encoder): LstmSeq2SeqEncoder(
    (_module): LSTM(300, 300, batch_first=True, bidirectional=True)
  )
  (_integrator): LstmSeq2SeqEncoder(
    (_module): LSTM(1800, 300, batch_first=True, bidirectional=True)
  )
  (_integrator_dropout): Dropout(p=0.1, inplace=False)
  (_self_attentive_pooling_projection): Linear(in_features=600, out_features=1, bias=True)
  (_output_layer): Maxout(
    (_linear_layers): ModuleList(
      (0): Linear(in_features=2400, out_features=4800, bias=True)
      (1): Linear(in_features=1200, out_featu

In [4]:
pprint(bcn.finetune_report())

{'best_epoch': 4,
 'best_validation_accuracy': 0.44595821980018163,
 'best_validation_accuracy3': 0.9028156221616712,
 'best_validation_loss': 1.214704468846321,
 'epoch': 4,
 'peak_gpu_0_memory_MB': 741.08740234375,
 'peak_worker_0_memory_MB': 2483.55859375,
 'training_accuracy': 0.4666432584269663,
 'training_accuracy3': 0.9103464419475655,
 'training_duration': '0:00:58.603155',
 'training_epochs': 4,
 'training_gpu_0_memory_MB': 741.08740234375,
 'training_loss': 1.2024494160053343,
 'training_start_epoch': 0,
 'training_worker_0_memory_MB': 2483.55859375,
 'validation_accuracy': 0.44595821980018163,
 'validation_accuracy3': 0.9028156221616712,
 'validation_loss': 1.214704468846321}


Prediction

In [5]:
print(f'Individual prediction for {data.NAME}')
bcn.predict(test.sentence[0])
print(bcn.predict(test[data.SENTENCE][0]))

Individual prediction for sst
     logits  class_probabilities label
0  0.034517             0.033491     1
1  2.991932             0.644636     1
2  1.353852             0.125287     1
3 -2.566013             0.002486     1
4  1.791625             0.194101     1


In [6]:
print(f'Batch prediction for {data.NAME}')
bcn.predict_batch_df(test[:100], input_col=data.SENTENCE)
print(bcn.predict_batch(test[data.SENTENCE][:100]))

Batch prediction for sst
                                               logits  \
0   [0.034516751766204834, 2.991931676864624, 1.35...   
1   [0.9285715818405151, 0.12953080236911774, 0.31...   
2   [0.590441882610321, 0.6019821763038635, 0.5148...   
3   [1.258155107498169, -0.06823420524597168, 0.30...   
4   [2.3645896911621094, -0.9855415225028992, 0.38...   
..                                                ...   
95  [1.750241756439209, 0.17843835055828094, 0.812...   
96  [0.22618433833122253, 1.1369720697402954, 0.52...   
97  [2.107375144958496, -0.674547016620636, 0.2986...   
98  [1.2020478248596191, 0.6326577663421631, 0.826...   
99  [2.7946228981018066, -0.523309588432312, 0.496...   

                                  class_probabilities label  
0   [0.03349077329039574, 0.6446353793144226, 0.12...     1  
1   [0.34913426637649536, 0.15702669322490692, 0.1...     3  
2   [0.25845009088516235, 0.2614499628543854, 0.23...     1  
3   [0.3808445930480957, 0.101089082658290

In [7]:
print(f'Individual label prediction for {data.NAME}')
print(bcn.predict_label(test.sentence[0]))
print(f'Batched label prediction for {data.NAME}')
bcn.predict_label_batch_df(test[:100], input_col=data.SENTENCE)
print(bcn.predict_label_batch(test.sentence[:100]))

Individual label prediction for sst
[1]
Batched label prediction for sst
[1 3 1 3 4 1 3 1 4 4 4 3 4 1 3 1 4 3 4 4 4 4 3 3 3 3 3 3 3 1 4 1 1 3 1 3 3
 3 1 1 3 1 4 4 3 1 4 3 3 1 3 1 3 4 3 1 3 1 4 4 3 3 3 3 3 3 3 4 4 3 3 3 4 1
 3 1 3 4 3 3 3 3 3 4 3 3 4 3 4 1 3 1 1 1 3 3 1 4 3 3]


In [8]:
print(f'Individual proba prediction for {data.NAME}')
print(bcn.predict_proba(test.sentence[0]))
print(f'Batched label prediction for {data.NAME}')
bcn.predict_proba_batch_df(test[:10], input_col=data.SENTENCE)
print(bcn.predict_proba_batch(test.sentence[:10]))

Individual proba prediction for sst
0    0.033491
1    0.644636
2    0.125287
3    0.002486
4    0.194101
Name: class_probabilities, dtype: float64
Batched label prediction for sst
[[0.03349077 0.64463538 0.12528679 0.00248616 0.19410087]
 [0.34913424 0.15702669 0.18819214 0.23571308 0.06993384]
 [0.25845012 0.26144999 0.23963159 0.12203951 0.11842884]
 [0.38084459 0.10108908 0.14691767 0.33236814 0.03878046]
 [0.23479119 0.00823642 0.03241548 0.7219243  0.00263267]
 [0.09286905 0.43066958 0.16412106 0.02648743 0.28585279]
 [0.40299612 0.14749025 0.19421567 0.2055884  0.0497094 ]
 [0.13400227 0.45397612 0.25728235 0.02084267 0.13389654]
 [0.4178907  0.01270929 0.04130781 0.52515405 0.00293825]
 [0.33184361 0.03716639 0.08408372 0.53323418 0.01367201]]


In [9]:
print(f'Individual logits prediction for {data.NAME}')
print(bcn.predict_logits(test.sentence[0]))
print(f'Batched logits prediction for {data.NAME}')
bcn.predict_logits_batch_df(test[:10], input_col=data.SENTENCE)
print(bcn.predict_logits_batch(test.sentence[:10]))

Individual logits prediction for sst
0    0.034517
1    2.991932
2    1.353852
3   -2.566013
4    1.791625
Name: logits, dtype: float64
Batched logits prediction for sst
[[ 0.03451669  2.99193192  1.35385239 -2.56601286  1.79162502]
 [ 0.92857146  0.12953082  0.3105785   0.53573024 -0.67933536]
 [ 0.59044188  0.60198224  0.51484191 -0.15991591 -0.18994853]
 [ 1.25815511 -0.06823425  0.3056362   1.12200689 -1.02631962]
 [ 2.36458969 -0.98554152  0.38452935  3.48781347 -2.12610912]
 [-0.11088881  1.42326188  0.45852506 -1.3654089   1.01339769]
 [ 1.24256682  0.23740195  0.51260912  0.569516   -0.8501662 ]
 [ 0.59167159  1.81185937  1.24398875 -1.26918304  0.59088218]
 [ 2.70833969 -0.78454745  0.39417142  2.93681145 -2.24906754]
 [ 1.68549609 -0.50376296  0.31264517  2.1597929  -1.50381696]]
